### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 25 ноября 2019, 15:00   
**Штраф за опоздание:** -2 балла после 15:00 25 ноября, -4 балла после 15:00 2 декабря, -6 баллов после 15:00 9 декабря  -8 баллов после 15:00 16 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0919, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) и [Speed Dating Data](https://cloud.mail.ru/public/8nHV/p6J7wY1y1)

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [44]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [45]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        return (np.sum(l_c ** 2) / l_s + \
                np.sum(r_c ** 2) / r_s) / (l_s + r_s) - \
                np.sum((l_c + r_c) ** 2) / ((l_s + r_s) ** 2)

    def __entropy(self, l_c, l_s, r_c, r_s):
        return (np.sum(l_c * np.log2(l_c / l_s)) + \
                np.sum(l_c * np.log2(l_c / l_s)) - \
                np.sum((l_c + r_c) * np.log2((r_c + l_c)) / \
                       (l_s + r_s))) / (l_s + r_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return (np.max(l_c) + np.max(r_c) - np.max(l_c + r_c)) / (l_s + r_s)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle([feature_ids])
        return np.asarray(feature_ids[:int(n_feature**0.5)])

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle([feature_ids])
        return np.asarray(feature_ids[:int(np.log2(n_feature))])

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        x.reshape(y.shape[0], -1)
        best_feature = 0
        max_gain = 0
        threshold = np.NINF
        b_gain_i = 0

        for i in self.get_feature_ids(x.shape[1]):
            x_sorted, y_sorted = self.__sort_samples(x[:, i], y)

            div_ind = np.where(x_sorted[:-1] != x_sorted[1:])[0]
            if div_ind.shape[0] == 0:
                continue

            div_ind += 1
            count1 = np.asarray(list(map(lambda i: np.bincount(y_sorted[:i:],
                                minlength=self.num_class), div_ind)))
            count2 = np.asarray(list(map(lambda i: np.bincount(y_sorted[i::],
                                minlength=self.num_class), div_ind)))
            gain_v = np.asarray(list(map(
                               lambda a, b, c, d: self.G_function(a, b, c, d),
                               count1, div_ind, count2, y.shape[0] - div_ind)))
            gain_i = np.argmax(gain_v)

            if max_gain <= gain_v[gain_i]:
                max_gain = gain_v[gain_i]
                best_feature = i
                threshold = x_sorted[gain_i]

        self.feature_importances_[best_feature] += max_gain * y.shape[0] / self.n_samples
        return best_feature, threshold

    def __fit_node(self, x, y, node_id, depth):
        class_, count_ = scipy.stats.mode(y)
        if (self.max_depth is not None and depth > self.max_depth) or \
            y.shape[0] < self.min_samples_split or \
            count_ / y.shape[0] >= self.sufficient_share:
            self.tree[node_id] = (self.__class__.LEAF_TYPE,
                                  class_, count_ / y.shape[0])
            return self.tree[node_id]

        feature_id, threshold = self.__find_threshold(x, y)
        x_l, x_r, y_l, y_r = self.__div_samples(x, y, feature_id, threshold)
        self.tree[node_id] = (self.__class__.NON_LEAF_TYPE,
                              feature_id, threshold)

        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)

        return self.tree[node_id]

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(shape=x.shape[1])
        self.n_samples = x.shape[0]
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11.4 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

CPU times: user 203 ms, sys: 0 ns, total: 203 ms
Wall time: 218 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

1.0

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

1.0

## Подготовка данных Speed Dating Data 

In [14]:
df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv', encoding='latin1')

In [15]:
df = df.iloc[:, :97]
df.shape

(8378, 97)

In [46]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 5
df

,iid,pid,match,int_corr,samerace,age,field_cd,mn_sat,tuition,race,imprace,imprelig,zipcode,income,goal,date,go_out,career_c,exphappy,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,iid_f,pid_f,age_f,field_cd_f,mn_sat_f,tuition_f,race_f,imprace_f,imprelig_f,zipcode_f,income_f,goal_f,date_f,go_out_f,career_c_f,exphappy_f,attr1_1_f,sinc1_1_f,intel1_1_f,fun1_1_f,amb1_1_f,shar1_1_f,attr2_1_f,sinc2_1_f,intel2_1_f,fun2_1_f,amb2_1_f,shar2_1_f,attr3_1_f,sinc3_1_f,fun3_1_f,intel3_1_f,amb3_1_f
3507,245,234.0,0,0.05,0,32.0,7.0,1430.0,26908.0,2.0,1.0,6.0,2173.0,0.0,4.0,5.0,2.0,1.0,2.0,18.0,10.0,30.0,10.0,10.0,22.0,25.0,11.0,11.0,18.0,20.0,15.0,5.0,7.0,6.0,8.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3508,245,235.0,0,0.43,1,32.0,7.0,1430.0,26908.0,2.0,1.0,6.0,2173.0,0.0,4.0,5.0,2.0,1.0,2.0,18.0,10.0,30.0,10.0,10.0,22.0,25.0,11.0,11.0,18.0,20.0,15.0,5.0,7.0,6.0,8.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8376,552,529.0,0,0.62,0,25.0,18.0,0.0,0.0,2.0,1.0,1.0,78110.0,0.0,1.0,2.0,1.0,15.0,10.0,70.0,0.0,15.0,15.0,0.0,0.0,50.0,0.0,0.0,30.0,0.0,20.0,8.0,7.0,6.0,7.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8377,552,530.0,0,0.01,0,25.0,18.0,0.0,0.0,2.0,1.0,1.0,78110.0,0.0,1.0,2.0,1.0,15.0,10.0,70.0,0.0,15.0,15.0,0.0,0.0,50.0,0.0,0.0,30.0,0.0,20.0,8.0,7.0,6.0,7.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
df = df.drop(['age_o', 'race_o', 'pf_o_att', 'pf_o_sin', 'pf_o_int','pf_o_fun', 'pf_o_amb', 'pf_o_sha','dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o','met_o'], 
             axis=1)
df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(19)
df = df.drop(['field'], axis=1)
df = df.drop(['undergra'], axis=1)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df['mn_sat'] = df['mn_sat'].fillna(value=0.0)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'zipcode'] = df.loc[:, 'zipcode'].str.replace(',', '').astype(np.float)
df['tuition'] = df['tuition'].fillna(value=0.0)
df = df.dropna(subset=['imprelig', 'imprace'])
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df['income'] = df['income'].fillna(value=0.0)
df = df.dropna(subset=['date'])
df = df.drop(['career'], axis=1)

In [21]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 5
df

,iid,gender,pid,match,int_corr,samerace,age,field_cd,mn_sat,tuition,race,imprace,imprelig,zipcode,income,goal,date,go_out,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,expnum,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,sinc5_1,intel5_1,fun5_1,amb5_1
0,1,0,11.0,0,0.14,0,21.0,1.0,0.0,0.0,4.0,2.0,4.0,60521.0,69487.0,2.0,7.0,1.0,NaN,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,0,12.0,0,0.54,0,21.0,1.0,0.0,0.0,4.0,2.0,4.0,60521.0,69487.0,2.0,7.0,1.0,NaN,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8376,552,1,529.0,0,0.62,0,25.0,18.0,0.0,0.0,2.0,1.0,1.0,78110.0,0.0,1.0,2.0,1.0,15.0,8.0,2.0,5.0,10.0,10.0,10.0,7.0,1.0,9.0,8.0,3.0,7.0,9.0,10.0,10.0,7.0,3.0,10.0,NaN,70.0,0.0,15.0,15.0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,10.0,50.0,0.0,0.0,30.0,0.0,20.0,8.0,7.0,6.0,7.0,7.0,9.0,7.0,10.0,5.0,9.0
8377,552,1,530.0,0,0.01,0,25.0,18.0,0.0,0.0,2.0,1.0,1.0,78110.0,0.0,1.0,2.0,1.0,15.0,8.0,2.0,5.0,10.0,10.0,10.0,7.0,1.0,9.0,8.0,3.0,7.0,9.0,10.0,10.0,7.0,3.0,10.0,NaN,70.0,0.0,15.0,15.0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,10.0,50.0,0.0,0.0,30.0,0.0,20.0,8.0,7.0,6.0,7.0,7.0,9.0,7.0,10.0,5.0,9.0


In [22]:
df['career_c'] = df['career_c'].fillna(value=0.0)
df = df.drop(['sports','tvsports','exercise','dining',
              'museums','art','hiking','gaming',
              'clubbing','reading','tv','theater','movies',
              'concerts','music','shopping','yoga'], axis=1)
df = df.drop(['expnum'], axis=1)
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 
                                        'intel1_1', 'fun1_1', 
                                        'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
(df.loc[:, ['attr1_1', 'sinc1_1',
            'intel1_1', 'fun1_1', 
            'amb1_1', 'shar1_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1',
                                        'intel2_1', 'fun2_1',
                                        'amb2_1', 'shar2_1']].sum(axis=1)

In [23]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 5
df

,iid,gender,pid,match,int_corr,samerace,age,field_cd,mn_sat,tuition,race,imprace,imprelig,zipcode,income,goal,date,go_out,career_c,exphappy,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,sinc5_1,intel5_1,fun5_1,amb5_1,temp_totalsum
0,1,0,11.0,0,0.14,0,21.0,1.0,0.0,0.0,4.0,2.0,4.0,60521.0,69487.0,2.0,7.0,1.0,0.0,3.0,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN,100.0
1,1,0,12.0,0,0.54,0,21.0,1.0,0.0,0.0,4.0,2.0,4.0,60521.0,69487.0,2.0,7.0,1.0,0.0,3.0,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8376,552,1,529.0,0,0.62,0,25.0,18.0,0.0,0.0,2.0,1.0,1.0,78110.0,0.0,1.0,2.0,1.0,15.0,10.0,70.0,0.0,15.0,15.0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,10.0,50.0,0.0,0.0,30.0,0.0,20.0,8.0,7.0,6.0,7.0,7.0,9.0,7.0,10.0,5.0,9.0,100.0
8377,552,1,530.0,0,0.01,0,25.0,18.0,0.0,0.0,2.0,1.0,1.0,78110.0,0.0,1.0,2.0,1.0,15.0,10.0,70.0,0.0,15.0,15.0,0.0,0.0,90.0,0.0,0.0,0.0,0.0,10.0,50.0,0.0,0.0,30.0,0.0,20.0,8.0,7.0,6.0,7.0,7.0,9.0,7.0,10.0,5.0,9.0,100.0


In [24]:
df.loc[:, ['attr2_1', 'sinc2_1',
           'intel2_1', 'fun2_1',
           'amb2_1', 'shar2_1']] = \
(df.loc[:, ['attr2_1', 'sinc2_1',
            'intel2_1', 'fun2_1',
            'amb2_1', 'shar2_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)
df = df.dropna()
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i), 
            'intel{}_1'.format(i), 'fun{}_1'.format(i), 
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    
    df = df.drop(feat, axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1)\
                                 .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                                   .dropna()
        
df_female.columns = df_female.columns + '_f'
df = df_male.join(df_female)
df = df.fillna(value=0)

X_df = df.drop(['match'], axis=1)
y_df = df['match']


In [26]:
X = np.array(X_df)
y = np.array(y_df)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2019)

In [27]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(1789, 68) (1789,) (199, 68) (199,)


## Проверка скорости работы на Speed Dating Data 

In [28]:
%time clf.fit(X_train, y_train)

CPU times: user 31.2 ms, sys: 0 ns, total: 31.2 ms
Wall time: 25.5 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [29]:
%time my_clf.fit(X_train, y_train)

CPU times: user 2.73 s, sys: 0 ns, total: 2.73 s
Wall time: 2.77 s


## Проверка качества работы на Speed Dating Data

In [31]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5211229946524064

In [32]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5216346153846154

## Задание 3

In [36]:
print("значения лучших фичей clf:", -np.sort(-clf.feature_importances_)[:10:])
best_features = np.argsort(-clf.feature_importances_)[:10:]
print("лучшие фичи clf:", best_features)

значения лучших фичей clf: [0.37478706 0.34567363 0.04630909 0.02536411 0.02236151 0.02095999
 0.0199396  0.01251075 0.01126427 0.01082312]
лучшие фичи clf: [ 1  2  3 19 11 23 30 10  7 20]


In [37]:
print("значения лучших фичей my_clf:", -np.sort(-my_clf.feature_importances_)[:10:])
best_features = np.argsort(-my_clf.feature_importances_)[:10:]
print("лучшие фичи my_clf:", best_features)

значения лучших фичей my_clf: [0.12342555 0.10083171 0.02364086 0.01995375 0.01842284 0.01777378
 0.00617001 0.00521152 0.00506156 0.00504844]
лучшие фичи my_clf: [ 2  1 34  3 21 26 28 24 23 29]


## Задание 4

In [47]:
param = {
    'criterion': ['gini', 'entropy'],'max_depth': [5, 10, 20, 40, 80],'min_samples_leaf': [2, 4, 8, 16, 32]}


model = DecisionTreeClassifier()
random_search = RandomizedSearchCV(model, param_distributions=param)
random_search.fit(X, y)

/home/evgenij_kosarev/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=DecisionTreeClassifier(class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort=False,
                                                    random_state=None,
                                                    splitter='b

In [39]:
random_search.best_params_

{'min_samples_leaf': 16, 'max_depth': 10, 'criterion': 'gini'}

In [40]:
random_search.best_score_

0.8455734406438632

In [41]:
model = DecisionTreeClassifier()
random_search = GridSearchCV(model, param_grid=param)
random_search.fit(X_train, y_train)

/home/evgenij_kosarev/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 10, 20, 40, 80],
             

In [42]:
random_search.best_params_

{'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 32}

In [43]:
random_search.best_score_

0.8501956400223588

## Задание 5